In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

In [2]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [4]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(feature_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer with ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        # Second GCN layer (no activation as we'll use cross entropy loss)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [6]:
model = GCN(dataset.num_node_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [7]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask]).backward() # negative likelihood loss
    optimizer.step()

# cross entropy loss for classification

def test():
    model.eval()
    logits = model(data)
    pred = logits.argmax(dim=1)
    correct_train = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
    correct_test = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    return correct_train / data.train_mask.sum().item(), correct_test / data.test_mask.sum().item()

In [8]:
# Load the dataset into data object
data = dataset[0]

In [9]:
# Training and testing the model
for epoch in range(200):
    train()
    train_acc, test_acc = test()
    print(f"Epoch {epoch+1}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch 1, Train Acc: 0.7429, Test Acc: 0.4280
Epoch 2, Train Acc: 0.8929, Test Acc: 0.5620
Epoch 3, Train Acc: 0.9143, Test Acc: 0.6270
Epoch 4, Train Acc: 0.9143, Test Acc: 0.6650
Epoch 5, Train Acc: 0.9357, Test Acc: 0.7050
Epoch 6, Train Acc: 0.9714, Test Acc: 0.7280
Epoch 7, Train Acc: 0.9714, Test Acc: 0.7500
Epoch 8, Train Acc: 0.9786, Test Acc: 0.7650
Epoch 9, Train Acc: 0.9786, Test Acc: 0.7690
Epoch 10, Train Acc: 0.9786, Test Acc: 0.7800
Epoch 11, Train Acc: 0.9857, Test Acc: 0.7850
Epoch 12, Train Acc: 0.9857, Test Acc: 0.7900
Epoch 13, Train Acc: 0.9857, Test Acc: 0.7880
Epoch 14, Train Acc: 0.9857, Test Acc: 0.7890
Epoch 15, Train Acc: 0.9857, Test Acc: 0.7920
Epoch 16, Train Acc: 0.9857, Test Acc: 0.7920
Epoch 17, Train Acc: 0.9929, Test Acc: 0.7920
Epoch 18, Train Acc: 0.9929, Test Acc: 0.7920
Epoch 19, Train Acc: 1.0000, Test Acc: 0.7940
Epoch 20, Train Acc: 1.0000, Test Acc: 0.7930
Epoch 21, Train Acc: 1.0000, Test Acc: 0.7930
Epoch 22, Train Acc: 1.0000, Test Acc: 0.79